In [2]:
%pip install seaborn

import pandas as pd

import seaborn as sns

df = sns.load_dataset('titanic')

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 7.9 MB/s eta 0:00:008.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 7.7 MB/s eta 0:00:00 MB/s eta 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
print("데이터 미리보기:")
print(df.head())

print("\n데이터 구조:")
print(df.info())
print('\n기본 통계 요약:')
print(df.describe())

데이터 미리보기:
   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  

데이터 구조:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   surv

In [4]:
stats = df.describe().to_string()

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(
  model_name="gpt-3.5-turbo",
  temperature=0
)

In [7]:
# Zero Shot
prompt_template = PromptTemplate(
    input_variables=["stats"],
    template=(
        "다음은 타이타닉 데이터셋의 기본 통계 정보입니다:\n{stats}\n\n"
        "이 데이터를 기반으로 전체 데이터의 주요 특징과 인사이트를 포함한 데이터 분석 보고서를 작성해 주세요."
    )
)
chain = LLMChain(llm=llm, prompt=prompt_template)
report_zero_shot = chain.run(stats=stats)

/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_90761/1856630363.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
/var/folders/1x/st3vh8xs6715dcgqc1gk2hhh0000gn/T/ipykernel_90761/1856630363.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  report_zero_shot = chain.run(stats=stats)


In [9]:
# Few Shot
few_shot_examples = """
예시 1:
데이터 통계:
         fare      age  survived
count  891.0  891.000  891.000
mean    32.20   29.70    0.383
std     49.70   14.53    0.486
min      0.00    0.42    0.000
max    512.33   80.00    1.000

[단계별 분석]
1. 요금(fare)의 분포가 넓어 일부 극단치가 존재할 수 있음.
2. 연령(age) 변수의 분포와 결측치 여부 확인 필요.
3. 생존률(survived)이 약 38%로, 전반적으로 낮은 편임.

[최종 보고서]
타이타닉 데이터셋은 요금 변수에서 극단치가 존재하며, 연령 변수에 대한 추가적인 결측치 처리와 분석이 필요합니다. 전반적으로 생존률이 낮은 점을 고려하여 추가적인 변수 분석 및 전처리가 요구됩니다.
----------------------------------

예시 2:
데이터 통계:
         fare      age  survived
count  891.0  891.000  891.000
mean    35.00   30.00    0.400
std     50.00   15.00    0.490
min      0.00    0.50    0.000
max    500.00   80.00    1.000

[단계별 분석]
1. 'fare' 변수의 평균과 표준편차를 통해 승객 요금의 변동성을 확인.
2. 'age' 변수에 대해 분포와 결측치를 검토하여 연령대별 특징 파악.
3. 'survived' 변수를 통해 전체 생존률 및 잠재적 예측 요소 도출.

[최종 보고서]
타이타닉 데이터셋은 승객들의 요금 분포와 연령대에 큰 변동성을 보입니다. 생존률은 약 40%로, 데이터의 결측치와 이상치 처리 후 추가적인 분석이 필요하며, 이를 통해 예측 모델 개선이 기대됩니다.
----------------------------------
"""

final_template = (
    f"{few_shot_examples}\n"
    "위의 예시들을 참고하여, 다음 타이타닉 데이터셋의 통계 정보를 기반으로 문제를 단계별로 분석하고, "
    "각 단계에서 고려한 사항들을 간략하게 설명한 후 최종 보고서를 작성해 주세요.\n\n"
    "데이터 통계 정보:\n{stats}"
)

prompt_template = PromptTemplate(
    input_variables=["stats"],
    template=final_template
)

chain = LLMChain(llm=llm, prompt=prompt_template)
report_few_shot = chain.run(stats=stats)

In [10]:
prompt_template = PromptTemplate(
    input_variables=["stats"],
    template=(
        "다음은 타이타닉 데이터셋의 기본 통계 정보입니다:\n{stats}\n\n"
        "이 데이터를 기반으로 전체 데이터의 주요 특징과 인사이트를 포함한 데이터 분석 보고서를 작성해 주세요.\n\n"
        "보고서를 작성하기 전에, 문제를 단계별로 분석하고 각 단계에서 고려한 사항들을 간략하게 설명해 주세요. "
        "예를 들어, 데이터의 분포, 결측치, 변수 간 상관관계, 생존률 등의 특징을 어떻게 해석할지 단계별로 기술한 후 최종 보고서를 작성해 주세요."
    )
)

chain = LLMChain(llm=llm, prompt=prompt_template)
report_cot = chain.run(stats=stats)

In [11]:
print("zero shot 생성된 데이터 분석 보고서:\n", report_zero_shot)
print('\n')
print('\n')
print("few shot 생성된 데이터 분석 보고서:\n", report_few_shot)
print('\n')
print('\n')
print("CoT 생성된 데이터 분석 보고서:\n", report_cot)

zero shot 생성된 데이터 분석 보고서:
 타이타닉 데이터셋은 총 891개의 승객 정보를 담고 있습니다. 생존율은 약 38.4%로 생존한 승객보다 사망한 승객이 더 많은 것으로 나타났습니다. 

승객들의 평균 나이는 약 29.7세이며, 나이의 표준편차는 약 14.5세로 분산이 크게 나타났습니다. 가장 어린 승객은 0.42세이고, 가장 나이 많은 승객은 80세입니다. 

승객들의 좌석 등급(pclass)은 1등급부터 3등급까지 다양하게 분포되어 있으며, 평균적으로는 2등급에 해당하는 승객이 많았습니다. 

가족과 함께 탑승한 승객의 수(sibsp, parch)는 평균적으로는 0.9명 정도이며, 대부분의 승객이 혼자 탑승했거나 소규모 가족과 함께 탑승한 것으로 나타났습니다. 

승객들이 지불한 요금(fare)은 평균적으로는 32.2달러이며, 최소 요금은 0달러이고 최대 요금은 512.33달러입니다. 대부분의 승객이 상대적으로 저렴한 요금을 지불한 것으로 나타났습니다. 

이를 통해 타이타닉 데이터셋을 통해 승객들의 인구통계학적 정보뿐만 아니라 생존율, 가족 구성원 수, 요금 등 다양한 정보를 파악할 수 있었습니다. 생존율과 관련하여 추가적인 분석을 통해 생존에 영향을 미치는 요인들을 파악할 수 있을 것으로 보입니다.




few shot 생성된 데이터 분석 보고서:
 [단계별 분석]
1. 생존 여부(survived) 변수의 평균을 통해 전체 생존률이 약 38%임을 확인.
2. 객실 등급(pclass) 변수의 분포를 통해 승객들의 등급 분포를 파악할 수 있음.
3. 연령(age) 변수의 결측치가 존재하며, 평균 연령은 약 29.7세임.
4. 형제자매/배우자 수(sibsp)와 부모/자녀 수(parch) 변수의 분포를 확인하여 가족 구성원에 대한 정보 파악.
5. 요금(fare) 변수의 평균과 표준편차를 통해 승객들의 지불 요금의 변동성을 확인.

[최종 보고서]
타이타닉 데이터셋은 생존률이 낮은 편이며, 연령 변수에 결측치가 존재하는 등 데이터 전처리가 필요합